Dataset

Training 
  Positive : 26974, 
  Negative : 27470

Validation 
  Positive : 2150, 
  Negative : 2136

In [ ]:
import json
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# from tensorflow import feature_column
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/2/training'

data-edit    training.txt	 Train_negative.txt   Train_positive.txt
train1.csv   training.xlsx	 Train_negative.xlsx
train1.xlsx  Train_negative.csv  Train_positive.csv


In [ ]:
#utitliy functions 

import os
import sys
# os.environ['MKL_THREADING_LAYER'] = 'GNU'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import datetime

output_path="/content/drive/MyDrive/Colab Notebooks/2/results/TargetNet_training/vgg_novel"

def set_output(string):
    """ set output configurations """
    output, save_prefix = sys.stdout, None
    if output_path is not None:
        save_prefix = output_path
        if not os.path.exists(save_prefix):
            os.makedirs(save_prefix, exist_ok=True)
        output = open( output_path + string + ".txt", "a")

    return output, save_prefix

def Print(string, output, newline=False, timestamp=True):
    """ print to stdout and a file (if given) """
    if timestamp:
        time = datetime.datetime.now()
        line = '\t'.join([str(time.strftime('%m-%d %H:%M:%S')), string])
    else:
        time = None
        line = string

    print(line, file=sys.stderr)
    if newline: print("", file=sys.stderr)

    if not output == sys.stdout:
        print(line, file=output)
        if newline: print("", file=output)

    output.flush()
    return time

In [ ]:
rol = []
for x in range(48):
  rol.append([0,0,0])
print(rol)
print(type(rol))



[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
<class 'list'>


In [ ]:
def encode_RNA(mirna_seq, mirna_esa, cts_rev_seq, cts_rev_esa,lenMirna):
    """ one-hot encoder for RNA sequences with/without extended seed alignments """

    mRnaseq = cts_rev_seq
    miRnaseq = mirna_seq
    mRnaesa = cts_rev_esa
    miRnaesa = mirna_esa

    seq_data = []
    seqrol = []

    for i in range(len(mRnaseq)):
        if i<5 or i>(len(miRnaseq)-1+5):
            rolTemp=[]
            for x in range(48):

              if x in range(4):
                if mRnaseq[i] == 'A':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(4,8):
                if mRnaseq[i] == 'U':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(8,12):
                if mRnaseq[i] == 'C':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(12,16):
                if mRnaseq[i] == 'G':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              elif x in range(16,20):
                if mRnaseq[i] == '-':
                    rolTemp.append([1,1,1])
                else:
                    rolTemp.append([0,0,0])
              else:
                rolTemp.append([0,0,0])

            # print("area1", np.array(rolTemp).shape)
            seq_data.append(rolTemp)
        else:
            if i<15:
                if mRnaesa[i-5] == 'A':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])


                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
                    # print("area2", np.array(rolTemp).shape)

                if mRnaesa[i-5] == 'U':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4,8):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])

                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    # print("area3", np.array(rolTemp).shape)
                    seq_data.append(rolTemp)
          
                if mRnaesa[i-5] == 'C':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(8,12):
                          rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                          

                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    # print("area4", np.array(rolTemp).shape)
                    seq_data.append(rolTemp)
                    

                if mRnaesa[i-5] == 'G':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(12,16):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    # print("area5", np.array(rolTemp).shape)
                    seq_data.append(rolTemp)

                if mRnaesa[i-5] == '-':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(16,20):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaesa[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaesa[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaesa[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaesa[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaesa[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    # print("area6", np.array(rolTemp).shape)
                    seq_data.append(rolTemp)
          
            else:
                if mRnaseq[i] == 'A':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                        # print("i is ",i)
                        # print("mi len is ",len(mirna_seq))
                        # print("i is ",i-5)
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaseq[i] == 'U':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(4,8):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
                
                if mRnaseq[i] == 'C':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(8,12):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaseq[i] == 'G':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(12,16):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)

                if mRnaseq[i] == '-':
                    rolTemp=[]
                    for x in range(48):
                        if x in range(16,20):
                            rolTemp.append([1,1,1])
                        elif x<20:
                          rolTemp.append([0,0,0])
                    
                        if x in range(20,24):
                          if miRnaseq[i-5] == 'A':
                             rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(24,28):
                          if miRnaseq[i-5] == 'U':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(28,32):
                          if miRnaseq[i-5] == 'C':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(32,36):
                          if miRnaseq[i-5] == 'G':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(36,40):
                          if miRnaseq[i-5] == '-':
                            rolTemp.append([1,1,1])
                          else:
                            rolTemp.append([0,0,0])
                        elif x in range(40,48):
                          rolTemp.append([0,0,0])

                    seq_data.append(rolTemp)
    
    for p in range(8):
      seq_data.append(rol)
    # print("data seq", np.array(seq_data).shape)
    x= np.array(seq_data)
    
    return x

In [ ]:
!pip install bio
import sys
import numpy as np
from Bio import pairwise2

# import torch

#Loading the datasets
Positive_Data = '/content/drive/MyDrive/Colab Notebooks/2/data/partial/miRAW_positive.csv'
Negative_Data = '/content/drive/MyDrive/Colab Notebooks/2/data/partial/miRAW_negative.csv'

TestingData = '/content/drive/MyDrive/Colab Notebooks/2/data/csv/Independent_dataset1.csv'

class miRNA_CTS_dataset():
    """ Pytorch dataloader for miRNA-CTS pair data """
    def __init__(self, X, labels, set_idxs, set_labels):
        self.X = X
        self.labels = labels
        self.set_idxs = set_idxs
        self.set_labels = set_labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.X[i], self.labels[i], self.set_idxs[i]

        

def reverse(seq):
    """ reverse the given sequence """
    seq_r = ""
    for i in range(len(seq)):
        seq_r += seq[len(seq) - 1 - i]
    return seq_r


score_matrix = {}  # Allow watson-crick & wobble
for c1 in 'ACGU':
    for c2 in 'ACGU':
        if (c1, c2) in [('A', 'U'), ('U', 'A'), ('G', 'C'), ('C', 'G')]:
            score_matrix[(c1, c2)] = 1
        elif (c1, c2) in [('U', 'G'), ('G', 'U')]:
            score_matrix[(c1, c2)] = 1
        else:
            score_matrix[(c1, c2)] = 0


def extended_seed_alignment(mi_seq, cts_r_seq):
    """ extended seed alignment """
    alignment = pairwise2.align.globaldx(mi_seq[:10], cts_r_seq[5:15], score_matrix, one_alignment_only=True)[0]
    mi_esa = alignment[0]
    cts_r_esa = alignment[1]
    esa_score = alignment[2]
    return mi_esa, cts_r_esa, esa_score


def get_dataset_from_configs(data_cfg,split_idx=None):
  
    FILE = open(data_cfg, "r")
    lines = FILE.readlines()
    FILE.close()

    X, labels, set_idxs, set_labels = [], [], [], []
    set_idx = 0
    for l, line in enumerate(lines[1:]):
      tokens = line.strip().split(",")
      print(tokens)
      mirna_id, mirna_seq, mrna_id, mrna_seq = tokens[:4]
      label = float(tokens[4]) if len(tokens) > 4 else 0
      
      mirna_seq = mirna_seq.upper().replace("T", "U")
      mrna_seq = mrna_seq.upper().replace("T", "U")
      mrna_rev_seq = reverse(mrna_seq)

      for pos in range(len(mrna_rev_seq) - 40 + 1):
          mirna_esa, cts_rev_esa, esa_score = extended_seed_alignment(mirna_seq, mrna_rev_seq[pos:pos+40])
          if esa_score < 6: continue
          X.append((encode_RNA(mirna_seq, mirna_esa,
                                                mrna_rev_seq[pos:pos+40], cts_rev_esa,len(mrna_seq))))
          labels.append(tf.expand_dims(np.array(label), 0))
          set_idxs.append(tf.expand_dims(np.array(set_idx), 0))

      set_labels.append(label)
      set_idx += 1
      if set_idx % 5 == 0:
          print('# {} {:.1%}'.format(split_idx, l / len(lines[1:])), end='\r', file=sys.stderr)
    print(' ' * 150, end='\r', file=sys.stderr)

    dataset = miRNA_CTS_dataset(X, labels, set_idxs, np.array(set_labels))
      
    return X, labels, set_idxs, set_labels

output, save_prefix = set_output("train_model_log")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 273 kB 14.3 MB/s 
     |████████████████████████████████| 2.7 MB 80.5 MB/s 


In [ ]:
start = Print(" ".join(['start loading datasets']), output)

test_X, test_y, test_set_idxs, test_set_labels = get_dataset_from_configs(TestingData)


12-08 15:13:11	start loading datasets


['hsa-miR-140-3p', 'GGCACCAAGAUGGGACACCAU', 'NM_007275', 'CCCTGGGAGGTGGCAGACAGAAGCACCCCCTGCCCCGGCAAGAAACTCCCAGGCTCAATCAAGGTGTGGCTTCCATTGAGGAGCCCAGGCTGGGGCCACAACCCTGAATAAACTCTGTTGGCCCATAACCTTCAGCTGTGAGCGGGTCGGTCCCACAGTATTGGTTGGGTGTTGGTTTGTGTGTGGACAAGAGGTGGTTGGTGGGTGGTGAAGGCTAATGGCAGAGTTAGCACCCCACTCTCCCAAGCCACCCCTGCAAGCAGCACAGCAGGGCATATACCAGTCAGGAATGCCCGTTACCTGGTTCCTTGCCTGGTCTGCTTTCTTCCAAGTTTGCCTGGGGCCTAGCCCTGCTAGAGGCTACAGCACTTTACAAGCAAGGTATGCTTTCTTCCAGCCCCTAGGCTGTGGGCACTGTATACAAGTAGGAACTTCCTTTCCTTCACTTCCCTTTTAACCCCTAGTCAGAGCATTTCAGCCGTTTGCTACCTCGATTCCTCCTGTGTTGGACAGAGGCTGGGGGCAGTGCCAGCCTGATTCTTCCGACCTACCTGCCATTTGTTCCCGCCTTCAGATGGATGGACAGTTTGCTGGCTATTGATAGGAGTGGGGACTGGGTGGGGGCTTCTCCCTCTACCCAGGGCTGGGCTGATCCCCCTACTGCAACTAACTGTTGCCCCCCAACCCCGAACCCCCAGTTGAGGAGTTGAGAGAGTGCAGGCTGGGGTCAGGACAGGCTGCGGATGCTTGTGCCTATGGGGAGTTACTCCAACCCACCTATTCTGTCTAATCTCCATGGCTTTGCACCAAATCCTCCACCCCTCCAATTGGGAGGGGACTGTTCACCACCTTGTGGTAAGGGACAACACCCTAAGGCTGGTGCCAGTAGTTATGAGTAGCCTACCACCCCCTCCCTTACAGTAACCCCCACCCCTTCAGGAT

['hsa-miR-140-3p', 'GGCACCAAGAUGGGACACCAU', 'NM_194291', 'TCCTCTTAGAATACCTATAAAAAGATGTAAACTAATGTACCTCAGTAATTAAATATGCTGTCACAACATTTAGGAATTAAGACAGTAACAGTATAGATATGGGATCAAATAATTTAGCATGTATTATGGAAAACACTAACTTATTGTGGCTTGATCTTCTTAGGACATCTTTTTTAAAAAGCTGTTTAGTATCATTTTGTGTATATTGTTGAAATGCTTTTTCATCAATAGCAGTCAACATTTTATCCTTTCTTTTTATATTCATAATGTTATTTAAGTGTCATTGATGTACTGTATTGACTTGGGGTTTGCTTATTTGTTACTTAACATGTGTACATGCATGAAAGCATTTTTCGTTGTTCCCTGATAGTTACATTTCAACCTTGGGATTTTTCCAAATTACTTAAGATGTTTAATGTCAGTTAAAGATTTTTTTACCCTCTTTTTGGGAACATCAATTTTGTACTGTTATGCAGTAAACATTTATAATAATATAATTTCAGTCATTTCTTAACTGTCACATCTATTGAAAATGGATATAGATACAGGTTTTAAGTATTTTAAGTATATATTACTTATTTTAATTTTCTGACTTTACTATTTTAAGGGCCAGAGGGTTAATCACAAAGAGCAATTATGTGGTCTCCCTGCTACATGAAACCGTGTATACTAACAAGCGTACAATTTTTAGTTGATTTTTTTTAACCTTTTAGTTTCCCAGTTTTGAATAATTACATGGTGGATTCTGACTTTTGAGGGGAAGCAAATGATTATTTTAGAGTCTTTGAAATGGGGATTGTGGAATTAGATTGAACTAAGGGATTTAACATGATGCTTGGAAATTAAGAGACTAAAGCTTTTTTTAAAAAAAGGTGGAAAATAGGAACTGTCAAGAAGGTTTATGGTATAAATGATGAAGTTGAAGTGATGTTTGAAAGATTA

['hsa-miR-107', 'ACUAUCGGGACAUGUUACGACGA', 'NM_001272004', 'CTTCCTCCGAGTGGAACTGCAGCCTGGGGACTTGATTAGGTGCTATGCATCAGTAGCATTTTGAATGCAAGGGATGATGGAAAGCAGCACATGCGTTTCTGTGGCAGCTGTGGGGACTTGACAGCAATGCTCATCTCTCATGCTTCAATTTTTCTTTTCTTACTGTTAATAGGAAAAAAATCAACATCTGTAAATTAAGAATACAGCAATTATCTGTACAGTACTGCATATTTGTAATACCCTTGTATATATGTTACTTGAATACAGAAATGTTCATTTGTGATGCTTTGCACTTGGGGGTGGGGGGTGGGAGGGAAATCAATTGCAACAAAGAGTGTGAGATGTGAGATTGTATAGAGATGAAGTGTCATCACATCATGTGCATGGTGCGGAACCTGCTGTTTTATCTATTTATTGTGCCGTGTTTACAGTTTTTTGTACACTGTACCTTCATTGGTTCCTGTGCTGTAGTAAATGTGTTAGGTAGCTGTGGACTCCTTGGTATTTTGTAAATGGTATAACATAACTTGGTTCCCCTCTGGGTCCTTGAGTTTTCTGTGTATCATGTGAAAAAAAATGGTGACATACATACAGAATTTTACAAAAAAAAAAAAAAAAAAAAAGGCATCAGTTTTTTTAAAAATGGGGAATGTACTATTAAATGGGGATCTTCCTGGTCTACTATCATTAGGACAAGTAACAAGCTAAAAATGAAGTCTCTTGAATCTTTCCATCCCCAACTTGCCCACAAAGCTGTGGGTGGTTTCTGGTACTTAAAGCACTAATATTATGTTGCTGCTCTGCGAACAGCCCAGTAGTCCCATTTCCTTCCACACCAAAAAGTGTTAAATCAAGTATGCAAATGGAGTCCTTACAACTGGAGTTTATGTTGTCTTGCCCTTTTTTTAGCACAAAAAAAATTGTACTTTTTTATTGTCGA

['hsa-miR-331-3p', 'AAGAUCCUAUCCGGGUCCCCG', 'NM_001271851', 'TCTGAATCCCAGCGTCGGGGCTCTGTGCCAGCTTACTCTTCACTCCAGGGTCGGATGCCACGTGCTACAGGACATGGGAGCTGCTGCTTGTGGGAATCTGGTGCCTGTTCCACTAGAGACAAGGGGTAGAGTTTCTCATTTGGATGAAAACCCCTTCAACTGGTGGTGTACAACTGAAGCTACTATATCTTTTTTGAAAATGGCAAAAAAAAAAAAAAAAAATTCTGGAGACCACAGAACTCAAGTGTGTGTTTCTCCTCTTTTGGGTCCCCTTTAAGTAGTTGGGATATTTTGGACCTGGAGATAACACCAGTTACCCATCCTTACCAGGGAATGTTGCCATCAATTCCAGTTGAAAATAATAGAAAAGACTGAATTTTTATATGCTTCACTTAGGCTTTCATTTGAGTAGACTCTAAAAATTCTGCCTTGCTTAAGTTCTAACACTGCCTCTCAGATTTCAGTTTTGGACATTGCACAACTAAGACCTTTTAAATGCATTTGCTTGCTAACTCGGAAGACACATAGTCTGCAGCAAGACATTCCTATATTGAAGAAATGAGAGAAAATTTTATGCTGCATCAGGTGGAGAGCAAGGCTCAACGGTGGTTGCATTAGTTCCCTCGGAAGTATTGAAAAAACTTTGAAATGGAAGAAAATTTTTGCACCTATGTTCTGAGTACCAGATGTCTGGGTTCTTTCTTCTGCATTAGATAAATGATCATGCTCAGTGTAACAAAGGGAATTAAAAGTTTTCCCACAGTCCCCTTCTAGGGGAGAAACTCATTGTGTCACTGAAATGTTTAGCTTACTTTAATCTTGATATCAGCCTCTACAGACCTTTTATTCTAAGCTATCGAGCTTCTTGATTGCATTTGGTTGACCACGAGTGACCCTAAGGCATTGGGGGACTGTCCACTGGGAGTTGGAGTGAGCACG

['hsa-miR-140-3p', 'GGCACCAAGAUGGGACACCAU', 'NM_001145909', 'AACTCTGTCCCACCAATCAGGAAGTCAAGAGGCTTCTGGCCCGCGTAGAAGAGGAGTGCAAACAACTCCAGAGGAGTCAACAGCAAAAACAGCAGGGCCCGCTACCAGCTCCACTCAACGACTCCGAGAACGAAGAGGACACCCCAACCCCTGGCTTAAGTGACCACTTTCACTCTGAGGAGACTGAAGAGGAAGAAACTTCTCCCCAGGAAGAATCTGTTTCCCCAACTCCCAGGTCCCAGCCATCCTCATCTGTCCCTTCCTCATACATCCGAAACCTTCAAGAAGGGTTACAGTCCAAAGGAAGGCCGGTATCGCCACAGAGCAGGGCAGGAATCGGCAAGTCCCTGAGAGAGCCTGTGGCCCAGCCAGGGCTGCTCCTGCAGCCCTCCAAGCAGGCCCAGATCGTGAAAACCAGCCAGCACCTGGGCTCTGGCCAGTCGGCAGTGAGAAATGGCAGTATGAAAGTTCAGATCTCTTCTCAGAACCCTCCTCCAAGTCCCATGCCAGGGAGAATCGCTGCCACTCCTGCTGGGAGCAGAACCCAGCATTTAGAGGGAACAGGTACTTTCACTACAAGAGCTGGTTGTGGCCACTTTGGGGATCGGCTGGGCCCCAGCCAGAATGTCCGCCTGCAGTGTGGTGAGAATGGCCCTGCACACCCTTTACCAAGTAAGACGAAAACCACAGAGAGGCTTCTGTCTCATTCCTCCGTGGCTGTGGACGCAGCCCCTCCAAACCAAGGTGGGCTGGCGACCTGCAGCGACGTGCGACACCCAGCTTCCCTCACCAGCTCAGGCTCTTCTGGTTCTCCATCCAGCAGCATAAAGATGTCAAGTTCAACCAGTAGTTTGACTTCGAGCAGCAGTTTTTCAGATGGCTTCAAGGTCCAAGGACCAGATACTAGAATTAAAGACAAGGTTGTAACCCACGTTCAGA

In [ ]:
test_X = np.array(test_X)
test_y = np.array(test_y)
test_set_idxs = np.array(test_set_idxs)

In [ ]:
print(np.array(test_X).shape)
print(np.array(test_y).shape)

In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.keras import regularizers, Input, Model, Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, BatchNormalization, MaxPool1D, MaxPool2D, Flatten, Dropout, Dense, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.models import load_model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tf.compat.v1.disable_eager_execution()

OutputDir = '/content/drive/MyDrive/Colab Notebooks/Ongoing/Novel/main_models/validation'

temp_y = 0
temp_x = []

final_x = []
final_y = []

bestModel = load_model(OutputDir + "/model.h5" )
score = bestModel.evaluate(test_X, test_y, verbose=0)

result = bestModel.predict(test_X)
result = np.array(result)

count =0
idx =0

for i in range(len(test_set_idxs)):
  # print("id", test_set_idxs[i])
  if test_set_idxs[i] == idx:
    temp_x.append(result[i])
    temp_y =test_y[i]
  else:
    max = np.max(temp_x)
    final_x.append(max)
    final_y.append(temp_y)
    temp_x= []
    temp_y =0
    idx +=1

max = np.max(temp_x)
final_x.append(max)
final_y.append(temp_y)
final_x= np.array(final_x)
final_y= np.array(final_y)

print("length of final ", len(final_x))
print("length of final y ", len(final_y))

for i in range(len(final_x)):
  if final_x[i] > 0.5 :
    # print("1")
    count=count+1.0
  # else :
    # print("0")

print("Count", count)

